In [1]:
import pandas as pd
import numpy as np
import datetime
from extract import get_full_data

def lwmean(arr):
    return np.mean(arr[-7:])


def lwmax(arr):
    return np.max(arr[-7:])


def max_blw(arr):
    return np.max(arr[-21:-7])


def mean_blw(arr):
    return np.mean(arr[-21:-7])


def max_full(arr):
    return np.max(arr)


def summary_save(source, label, threshold=None):
    full_df = all_countries[all_countries['iso_code'].isin(source['iso_code'].unique())]
    full_df = full_df[full_df['date'] == today]
    full_df['case_per_millon'] = (1000000 * full_df['total_cases']) / full_df['population']
    full_df['tests_per_millon'] = (1000000 * full_df['total_tests']) / full_df['population']

    filtered = full_df[['iso_code', 'location', 'date', 'total_cases', 'new_cases', 'case_per_millon', 'tests_per_millon', 'total_tests', 'population']]
    if threshold is not None:
        filtered = filtered[filtered['case_per_millon'] > threshold]

    filtered.reset_index(inplace=True, drop=True)

    cap_mean = filtered['case_per_millon'].mean()
    cap_std = filtered['case_per_millon'].std()
    cap_max = filtered['case_per_millon'].max()
    cap_min = filtered['case_per_millon'].min()

    print(f"mean: {cap_mean}")
    print(f"std: {cap_std}")
    print(f"max: {cap_max}")
    print(f"min: {cap_min}")

    filtered.to_csv(f'data/{label}.csv', index=False)
    filtered.to_csv(f'data/date/{label}{today}.csv', index=False)
    return filtered

def get_summary(data, threshold=None):
    if threshold is not None:
        filtered = data[data['population'] > threshold]

    filtered = filtered[['iso_code', 'new_cases']]
    summary = filtered.groupby('iso_code').agg([np.mean, max_blw, lwmean, mean_blw])
    summary.columns = summary.columns.droplevel(0)
    summary['descending'] = summary['mean_blw'] > summary['lwmean']
    summary['closing'] = summary['descending'] & (summary['mean'] / 2 > summary['lwmean'])
    summary.reset_index(inplace=True)
    return summary


In [2]:
today_dt = datetime.date.today()
today = today_dt.strftime('%Y-%m-%d')
all_countries = get_full_data(force=True)
all_countries['cases_per_million'] = all_countries['total_cases'] / all_countries['population'] * 1000000

exclude_small = all_countries[all_countries['population'] > 500000]

all_countries.head()

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k,Entity,Year,Population,cases_per_million
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,2019,106000.0,18.732555
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,2019,106000.0,37.465111
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,2019,106000.0,112.395332
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,2019,106000.0,159.226720
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,NaN,11.62,NaN,NaN,NaN,NaN,Aruba,2019,106000.0,177.959275


### Cuales estan descendiendo o cerrando su curva?

In [3]:
summary = get_summary(all_countries, threshold=500000)
summary

,iso_code,mean,max_blw,lwmean,mean_blw,descending,closing
0,AFG,101.573427,1063,646.714286,426.071429,False,False
1,AGO,1.183099,6,3.428571,1.214286,False,False
2,ALB,13.357143,18,19.000000,9.500000,False,False
3,ARE,230.585034,994,741.714286,806.214286,True,False
4,ARG,188.383721,718,694.428571,398.357143,False,False
...,...,...,...,...,...,...,...
164,XKX,13.818182,35,5.571429,11.071429,True,True
165,YEM,5.961538,37,14.000000,12.714286,False,False
166,ZAF,269.278261,1218,1374.857143,851.642857,False,False
167,ZMB,14.283784,208,19.571429,47.714286,True,False


In [4]:
BOL = all_countries[all_countries['iso_code'] == 'BOL'][all_countries['date'] == today]['total_cases'].values[0]
bol_case_per_million = BOL / 11.63

print(f"BOL: {BOL}")
print(f"bol_case_per_million: {bol_case_per_million}")

BOL: 9592
bol_case_per_million: 824.7635425623387


/Users/jose.zambrana/miniconda/envs/covid19/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [17]:
if not summary[summary['iso_code'] == 'BOL']['descending'].values[0]:
    factor = 2
else:
    factor =1

In [18]:
worse = exclude_small[exclude_small['cases_per_million'] > (bol_case_per_million * factor)][exclude_small['date'] == today]
worse = worse.sort_values(['cases_per_million'], ascending=False)
worse = worse[['iso_code', 'location', 'date', 'total_cases', 'new_cases', 'cases_per_million', 'population']]
worse.reset_index(inplace=True, drop=True)
worst = worse.head(25)
worst.reset_index(inplace=True, drop=True)
worst.head(50)

/Users/jose.zambrana/miniconda/envs/covid19/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,iso_code,location,date,total_cases,new_cases,cases_per_million,population
0,QAT,Qatar,2020-05-31,55262,2355,19181.134721,2881060.0
1,LUX,Luxembourg,2020-05-31,4016,4,6415.581428,625976.0
2,BHR,Bahrain,2020-05-31,10793,344,6342.917154,1701583.0
3,KWT,Kuwait,2020-05-31,26192,1008,6133.149189,4270563.0
4,SGP,Singapore,2020-05-31,34366,506,5874.185496,5850343.0
5,USA,United States,2020-05-31,1770384,23297,5348.549373,331002647.0
6,IRL,Ireland,2020-05-31,24929,53,5048.608732,4937796.0
7,BEL,Belgium,2020-05-31,58186,125,5020.528722,11589616.0
8,CHL,Chile,2020-05-31,94858,4220,4962.176339,19116209.0
9,PER,Peru,2020-05-31,155671,7386,4721.331041,32971846.0


In [19]:
summary_save(worst, 'worst', threshold=bol_case_per_million)

mean: 4707.819553196989
std: 3266.7759116797006
max: 19181.13472124843
min: 2389.344096046233


,iso_code,location,date,total_cases,new_cases,case_per_millon,tests_per_millon,total_tests,population
0,ARE,United Arab Emirates,2020-05-31,33896,726,3427.161692,NaN,NaN,9890400.0
1,ARM,Armenia,2020-05-31,8927,251,3012.586924,NaN,NaN,2963234.0
2,BEL,Belgium,2020-05-31,58186,125,5020.528722,NaN,NaN,11589616.0
3,BHR,Bahrain,2020-05-31,10793,344,6342.917154,NaN,NaN,1701583.0
4,BLR,Belarus,2020-05-31,41658,894,4408.570732,NaN,NaN,9449321.0
5,CAN,Canada,2020-05-31,90179,772,2389.344096,NaN,NaN,37742157.0
6,CHE,Switzerland,2020-05-31,30762,17,3554.402979,NaN,NaN,8654618.0
7,CHL,Chile,2020-05-31,94858,4220,4962.176339,NaN,NaN,19116209.0
8,DJI,Djibouti,2020-05-31,3194,280,3232.786978,NaN,NaN,988002.0
9,GBR,United Kingdom,2020-05-31,272826,1604,4018.884364,NaN,NaN,67886004.0


### Que paises estan cerrrando o en decenso de su curva de contagio?

In [20]:
summary_save(summary[summary['descending']][summary['closing']], 'closing', threshold=1000)

mean: 3020.0861113367027
std: 1818.9903834468203
max: 6415.58142804197
min: 1076.6081477430616


/Users/jose.zambrana/miniconda/envs/covid19/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,iso_code,location,date,total_cases,new_cases,case_per_millon,tests_per_millon,total_tests,population
0,AUT,Austria,2020-05-31,16638,44,1847.352993,NaN,NaN,9006400.0
1,CHE,Switzerland,2020-05-31,30762,17,3554.402979,NaN,NaN,8654618.0
2,CYP,Cyprus,2020-05-31,943,2,1076.608148,NaN,NaN,875899.0
3,DEU,Germany,2020-05-31,181482,286,2166.071316,NaN,NaN,83783945.0
4,IRL,Ireland,2020-05-31,24929,53,5048.608732,NaN,NaN,4937796.0
5,ITA,Italy,2020-05-31,232664,416,3848.113888,NaN,NaN,60461828.0
6,LUX,Luxembourg,2020-05-31,4016,4,6415.581428,NaN,NaN,625976.0
7,NOR,Norway,2020-05-31,8411,0,1551.489493,NaN,NaN,5421242.0
8,SRB,Serbia,2020-05-31,11381,27,1672.546026,NaN,NaN,6804596.0


In [21]:
summary[summary['descending']]
summary_save(summary[summary['descending']], 'descending', threshold=1000)

mean: 3412.668564641401
std: 1721.052448037072
max: 6415.58142804197
min: 1076.6081477430616


,iso_code,location,date,total_cases,new_cases,case_per_millon,tests_per_millon,total_tests,population
0,ARE,United Arab Emirates,2020-05-31,33896,726,3427.161692,NaN,NaN,9890400.0
1,AUT,Austria,2020-05-31,16638,44,1847.352993,NaN,NaN,9006400.0
2,BEL,Belgium,2020-05-31,58186,125,5020.528722,NaN,NaN,11589616.0
3,BHR,Bahrain,2020-05-31,10793,344,6342.917154,NaN,NaN,1701583.0
4,BLR,Belarus,2020-05-31,41658,894,4408.570732,NaN,NaN,9449321.0
5,CAN,Canada,2020-05-31,90179,772,2389.344096,NaN,NaN,37742157.0
6,CHE,Switzerland,2020-05-31,30762,17,3554.402979,NaN,NaN,8654618.0
7,CYP,Cyprus,2020-05-31,943,2,1076.608148,NaN,NaN,875899.0
8,DEU,Germany,2020-05-31,181482,286,2166.071316,NaN,NaN,83783945.0
9,DNK,Denmark,2020-05-31,11633,40,2008.389554,NaN,NaN,5792203.0
